                                                                    بِسْمِ اللَّـهِ الرَّحْمَـٰنِ الرَّحِيمِ 


                                                                        Google Maps Scrapper

In [79]:
from selenium import webdriver
#import pyautogui
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


In [80]:
filename = "data4"
link = "https://www.google.com/maps/search/restaurants+in+El-Marg,+Al+Marg+Al+Qebleyah,+El+Marg/@30.1486887,31.3254932,16z/data=!4m2!2m1!6e5?entry=ttu" 
# browser = webdriver.Edge()
record = []
e = []
le = 0

driver=webdriver.Edge()
#driver.get("https://www.google.com/maps/search/restaurants+in+Cairo/@30.0467159,31.2205123,11.96z/data=!4m2!2m1!6e5?entry=ttu")
print(driver.title)


def Selenium_extractor():
    

    #Change language to English  
    langChangeStep1=driver.find_element(By.XPATH,""" //*[@id="QA0Szd"]/div/div/div[1]/div[1]/ul/li[1]/button """)
    langChangeStep1.click()
    time.sleep(2)
    langChangeStep2=driver.find_element(By.XPATH,"""//*[@id="settings"]/div/div[2]/ul/div[7]/li[1]/button""")
    langChangeStep2.click()
    time.sleep(2)
    langChangeStep3=driver.find_element(By.XPATH,""" //*[@id="languages"]/div/div[2]/div/div/a""")
    langChangeStep3.click()
    time.sleep(2)

    action = ActionChains(driver)
    a = driver.find_elements(By.CLASS_NAME, "hfpxzc")


    while len(a) < 5:
        print(len(a))
        var = len(a)
        scroll_origin = ScrollOrigin.from_element(a[len(a)-1])
        action.scroll_from_origin(scroll_origin, 0, 1000).perform()
        time.sleep(2)
        a = driver.find_elements(By.CLASS_NAME, "hfpxzc")

        if len(a) == var:
            le+=1
            if le > 20:
                break
        else:
            le = 0
    print("----------------------")
    print("    Scrolling Done    ")
    print("----------------------")
    
    
    for i in range(len(a)):
        scroll_origin = ScrollOrigin.from_element(a[i])
        action.scroll_from_origin(scroll_origin, 0, 100).perform()
        action.move_to_element(a[i]).perform()
        
        #This (try-except) is done to ensure that the element is loaded so its clickable or not to avoid any (element not clickable)error
        while(True):
            try:
                a[i].click()
                break
            except:
                continue

        time.sleep(2)
        source = driver.page_source
        soup = BeautifulSoup(source, 'html.parser')
        # try:
        
        #name
        Name_Html = soup.findAll('h1', {"class": "DUwDvf lfPIob"})
        name = Name_Html[0].text
        if name not in e:
            e.append(name)
            divs = soup.findAll('div', {"class": "Io6YTe fontBodyMedium kR99db"})
            
            
            #photos:
            try:
                time.sleep(2)
                photoLinkButton=driver.find_element(By.XPATH ,""" //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[1]/div[1]/button/img  """)
                photoLinkButton.click()
                
                action = ActionChains(driver)
                a_photos = driver.find_elements(By.XPATH, """ //div[@class="m6QErb "]//a """)
                print(len(a_photos))
                le = 0

                while len(a_photos) < 15:
                    print(len(a_photos))
                    var = len(a_photos)
                    scroll_origin = ScrollOrigin.from_element(a_photos[len(a_photos)-1])
                    action.scroll_from_origin(scroll_origin, 0, 1000).perform()
                    time.sleep(2)
                    a_photos = driver.find_elements(By.CLASS_NAME, "OKAoZd ")

                    if len(a_photos) == var:
                        le+=1
                        if le > 1:
                            break
                    else:
                        le = 0
                print("-----------------------------")
                print("    Photos Scrolling Done    ")
                print("-----------------------------")
                
                PhotoLinks=driver.find_elements(By.XPATH,""" //div[@class="U39Pmb"] """)
                counter=0
                for links in PhotoLinks:
                    photoLink=PhotoLinks[counter].get_attribute("style")[23:-3]
                    print(photoLink)
                    counter=counter+1
                    if(counter>12):
                        break   
            except:
                print("No photos")
                
                


                
            
            #Address
            Address=divs[0].text                                                                                       


            #phone number
            phoneFound=False
            
            while( phoneFound==False):
                for j in range(len(divs)):                                                                                 
                    for z in range(len(divs[j].text)):                                  #seeing every div from inside
                        if((str(divs[j].text)[z].isnumeric()) or (str(divs[j].text)[z] ==" ") or (str(divs[j].text)[z] == "+")):
                            if(z == (len(str(divs[j].text)))-1):                                           #does he tested all the char of the phone number or not
                                phone  = divs[j].text
                                phoneFound=True  
                            else:
                                continue
                        else:
                            if(phoneFound==False):
                                if(j == len(divs)-1):                                           # does he tested all the divs or not
                                    phone  = "not available"
                                    phoneFound=True                                            # to break the outer loop only (practically phone is not found )
                                else:  
                                    break
                             
                    
            #Website
            websiteFound=False                                                                               
            for z in range(len(divs)):
                if(len(divs[z].text)> 5):
                    if str(divs[z].text)[-4] == "." or str(divs[z].text)[-3] == "." or str(divs[z].text)[-5] == ".":         # www.  , .com , .site
                        if(str(divs[z].text)=="facebook.com") or (str(divs[z].text)=="m.facebook.com"):
                            website=driver.find_element(By.XPATH,""" //div[@class="RcCsl fVHpi w4vB1d NOE9ve M0S7ae AG25L "]/a  """).get_attribute("href")
                        else:
                            website = divs[z].text
                        websiteFound=True
                            
                    if(websiteFound==False):
                        website="Not available"
                # else:
                #     website="Not available"  
                #     print("not av")
        
            
            # rating
            Rating_Html = soup.findAll('div', {"class": "F7nice"})
            rating=Rating_Html[0].text[0:3]
            noOfRatings=Rating_Html[0].text[4:-1]
            
            
            #location link
            location_Html=driver.find_element(By.XPATH,""" //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[4]/div[5]/button """)
            location_Html.click()
            time.sleep(2)
            #the function of this loop is to wait until the link is loaded soo noo error occur while executing the code
            linkFound=False
            while(linkFound==False):
                try:
                # locationLink=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[3]/div/div/div[1]/div[4]/div[2]/input """).get_attribute("value")
                    locationLink=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[3]/div/div/div/div[3]/div[2]/div[2]/input """).get_attribute("value")
                    linkFound =True
                    break
                except:
                    time.sleep(1)
                    continue
                
            time.sleep(2)
            closeSharePanel=driver.find_element(By.XPATH,""" //*[@id="modal-dialog"]/div/div[2]/div/div[2]/button """)
            closeSharePanel.click()
            time.sleep(2)
            
            
            
            #plus code 
            counter=0
            plusCodeValue ="None"
            plusCodeFound=False   
            divPhotos=driver.find_elements(By.XPATH,"""//div[@class="cXHGnc"]""")    


            while(plusCodeFound==False):
                
                for z in range(len(divPhotos)):

                    t1="(//div[@class='cXHGnc']/div/img)["
                    t2=z+1
                    t3="]"
                    photoPath=str(t1)+str(t2)+str(t3)
                    Photo=driver.find_element(By.XPATH,str(photoPath) )
                    if(Photo.get_attribute("src")=="https://maps.gstatic.com/mapfiles/maps_lite/images/2x/ic_plus_code.png"):
                        plusCodeValue=divs[counter].text
                        plusCodeFound=True
                        break
                    else:
                        if(plusCodeFound==False):
                            if(z == len(divPhotos)-1):                                           # does he tested all the divs or not
                                plusCodeValue  = "not available"
                                plusCodeFound=True                                            # to break the outer loop only (practically phone is not found )
                            else:  
                                counter =counter+1
       
            p = driver.current_window_handle                #(save the last place in the main page)
        
            #Longitude and latitude (from plus code if found or from address if not found)
            plusCodeToLongLatWebsite ="https://plus.codes/map"
            # Open a new window
            driver.execute_script("window.open('');")
            # Switch to the new window and open the website
            driver.switch_to.window(driver.window_handles[1])
            driver.get(plusCodeToLongLatWebsite)
            time.sleep(2)
            
            inputPlusCode = driver.find_element(By.XPATH, """ //*[@id="search-input"] """)
            
            if(plusCodeValue != "not available"):
                inputPlusCode.send_keys(plusCodeValue)
                inputPlusCode.send_keys(Keys.ENTER)
            else:
                inputPlusCode.send_keys(Address)
                inputPlusCode.send_keys(Keys.ENTER)
            time.sleep(2)
            showLongitudeAndLatitudePanel=driver.find_element(By.XPATH,""" //*[@id="summary"]/div[1] """)
            showLongitudeAndLatitudePanel.click()
            
            time.sleep(2)
            
            longitudeAndLatitude =driver.find_element(By.XPATH,""" //*[@id="placecard-details"]/div[3]/div[1]  """).text
            latitude , longitude = longitudeAndLatitude.split(',')
            driver.close()
            
            driver.switch_to.window(p)
           
           
            print([name ,Address , photo, phone ,website , rating ,noOfRatings,locationLink,plusCodeValue ,latitude ,longitude])
            record.append((name ,Address ,photo , phone ,website ,rating ,noOfRatings ,"Cairo",locationLink ,plusCodeValue ,latitude ,longitude))
            df=pd.DataFrame(record,columns=['Name' ,'Address' ,'Photo link' ,'Phone number' ,'Website' ,'Rating','Number of ratings' ,"Governorate","location Link","Plus code" ,"Latitude" ,"Longitude"])  # writing data to the file
            df.to_csv(filename + '.csv',index=False,encoding='utf-8')
        # except:
        #     print("error")
        #     continue




driver.get(str(link))
time.sleep(10)
Selenium_extractor()



----------------------
    Scrolling Done    
----------------------
2
2
7
7
-----------------------------
    Photos Scrolling Done    
-----------------------------
https://lh5.googleusercontent.com/p/AF1QipMixA6-4ETU-UjsfKgZWl7MQp6KsLgipyNsbdpA=w203-h151-k-no
https://lh5.googleusercontent.com/p/AF1QipOsonUlP88JaLVlTCnciqbMiQnCdLU1w8TRy_Rm=w203-h152-k-no
https://lh5.googleusercontent.com/p/AF1QipOP7rCPKcEZwIJOjXlwYKEqHmJOsG6OE-y-zeMz=w203-h135-k-no
https://lh5.googleusercontent.com/p/AF1QipNQwhaHMqgjKhRNqmVlMu77UlXHQO7pvm4J6HEC=w203-h203-k-no
https://lh5.googleusercontent.com/p/AF1QipNwvA-y7QnknAoVkgJ14NQL8PkzLeEofMK8vOPG=w203-h193-k-no
https://lh5.googleusercontent.com/p/AF1QipPgukqnRV4N0ZI2-bIdxjyS_X2U2eKFCaYNi_sx=w203-h120-k-no
https://lh5.googleusercontent.com/p/AF1QipPnIe06-6BGBCFslgRmzCR9GoXUljzd1x5ixP-l=w203-h266-k-no


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":" //*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[4]/div[5]/button "}
  (Session info: MicrosoftEdge=118.0.2088.76); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF77A458942+63554]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF77A3E2462+257778]
	(No symbol) [0x00007FF77A1C1139]
	(No symbol) [0x00007FF77A206098]
	(No symbol) [0x00007FF77A2061A9]
	(No symbol) [0x00007FF77A23C717]
	(No symbol) [0x00007FF77A224C3F]
	(No symbol) [0x00007FF77A1FB541]
	(No symbol) [0x00007FF77A23A21B]
	(No symbol) [0x00007FF77A2249D3]
	(No symbol) [0x00007FF77A1FA574]
	(No symbol) [0x00007FF77A1F9C40]
	(No symbol) [0x00007FF77A1FACF4]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF77A618BC9+1121529]
